<img src="../common/fun_long_logo-01.png">

# Tutorial on making batch calculations using dataset from GEMS and adding your own data.
Miron et al., (2023). ThermoFun: A C++/Python library for computing standard thermodynamic properties of substances and reactions across wide ranges of temperatures and pressures. Journal of Open Source Software, 8(83), 4624

[![DOI](https://joss.theoj.org/papers/10.21105/joss.04624/status.svg)](https://doi.org/10.21105/joss.04624)

### Import ThermoFun module
ThermoFun delivers thermodynamic properties of substances and reactions at the temperature and pressure of interest.

In [ ]:
import thermofun as fun

### Initialize a ThermoFun database record - 
Thermofun needs a database of substances and or reactions. The data is in JSON format. Have a look at the json files '../databases/'. One can also use the available datasets in thermohub database and retrieve them uisng thermohubclient. ThermoHub contains the up to date datasets. 
Another possible source for ThermoFun database files is https://github.com/thermohub/thermohub mirror of thermohub database.

For this example we will use a dataset that comes from a GEMS project, containing a subset of elements (REE). For this you need to go in your GEMS project select the SysEq system you want to export and go to Data->Export GEMS3K files and tick FUN-json option. From the resulted files we need the one finishing with -fun.json

In [ ]:
# initalize a database using a local file HREE-P-OH_3K-fun.json
REE = fun.Database('gemsfiles/HREE-P-OH_3K-fun.json')

In [ ]:
# print available substances 
print(REE.mapSubstances().keys())

In [ ]:
# print avaialble reactions
print(REE.mapReactions().keys())

### Add your own data file
Adjust the gibbs_energy of Er+3 based on the fit result, add it as a record to mydata-thermofun.json with a new symbol Er+3_fit

In [ ]:
REE.appendData('myREEdata-thermofun.json')
#batch = fun.ThermoBatch(REE)

### Initialize a Batch class object
This object ca be used to do batch calculations for a list of substances or reactions, T-P points, and properties

In [ ]:
batch = fun.ThermoBatch(REE)

### inspect avialable batch functions. 
print(fun.availablePropertiesReaction())

In [ ]:
print(fun.availablePropertiesSubstance())
print(fun.availablePropertiesReaction())

Set units to C and bar, setPropertiesUnits(["temperature", "pressure"],["degC","bar"])

In [ ]:
batch.setPropertiesUnits(["temperature", "pressure"],["degC","bar"])
batch.setPropertiesDigits(["heat_capacity_cp","entropy", "temperature", "pressure"], [4, 4, 0, 2])

### set temperature and pressure increments

In [ ]:
batch.setPressureIncrement(0,0,0)
batch.setTemperatureIncrement(0,250, 5)

### Calculate properties and plot
"Er+3", "Sm+3", "Eu+3"
"heat_capacity_cp","entropy"

In [ ]:
substances = ["Er+3", "Sm+3", "Eu+3"]
properties = ["heat_capacity_cp","entropy", "volume"]
batch.thermoPropertiesSubstance(substances, properties).toCSV('results_substances.csv')

### inspect the results file and plot the results

In [ ]:
from common.functions import plot_substances_properties_vs_temperature
figure = plot_substances_properties_vs_temperature('results_substances.csv', substances, 0)

### Properties of reaction equation 

In [ ]:
engine = fun.ThermoEngine(REE)
reaction_properties = engine.thermoPropertiesReaction(348.15, 1e5, "ErPO4 = Er+3 + PO4-3")
logK = reaction_properties.log_equilibrium_constant
print(f'logK (ErPO4 = Er+3 + PO4-3) is {logK.val}')

### Batch calculation for multiple reactions

In [ ]:
batch.setPropertiesDigits(["logKr"], [4])
reactions = ["YPO4 = Y+3 + PO4-3", "SmPO4 = Sm+3 + PO4-3", "EuPO4 = Eu+3 + PO4-3", "rhabdophane-(Sm)"]
batch.thermoPropertiesReaction(reactions, ["logKr"]).toCSV("results_reactions.csv")

In [ ]:
from common.functions import plot_properties_vs_temperature
plt = plot_properties_vs_temperature('results_reactions.csv', reactions, 0)
plt.gca().invert_xaxis()
plt.show()

### compare with experiments 

logKsp of ErPO4 = Er+3 + PO4-3

<img style="float: center; height: 70%; width: 30%;" src="Xtm-Er-logKsp.png"> 

In [ ]:
reactions = ["ErPO4 = Er+3 + PO4-3"]
batch.thermoPropertiesReaction(reactions, ["logKr"]).toCSV("results_reactions.csv")

In [ ]:
import pandas as pd
df = pd.read_csv('loKsp_Er.csv')

plt = plot_properties_vs_temperature('results_reactions.csv', reactions, 0)

# Extract X and Y data
x_data = df['TC']
y_data = df['logKsp']

# Create the plot
plt.plot(x_data, y_data, 'X', color='r', markersize=15, markeredgecolor="w")
plt.gca().invert_xaxis()
plt.show()

### compare with the fit results "ErPO4 = Er+3_fit + PO4-3"

In [ ]:
reactions = ["ErPO4 = Er+3 + PO4-3", "ErPO4 = Er+3_fit + PO4-3"]
batch.thermoPropertiesReaction(reactions, ["logKr"]).toCSV("results_reactions.csv")

In [ ]:
import pandas as pd
df = pd.read_csv('loKsp_Er.csv')

plt = plot_properties_vs_temperature('results_reactions.csv', reactions, 0)

# Extract X and Y data
x_data = df['TC']
y_data = df['logKsp']

# Create the plot
plt.plot(x_data, y_data, 'X', color='r', markersize=15, markeredgecolor="w")
plt.gca().invert_xaxis()
plt.show()